In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as st
from collections import Counter
import matplotlib.pyplot as plt
from random import choices
from matplotlib.pyplot import figure
import mpl_scatter_density

In [3]:
year_analysis = {"0-1990": 0, "1990-1995": 0, "1995-2000": 0, "2000-2005": 0, "2005-2010": 0, "2010-2018": 0}

In [8]:
def update_year_analysis(file):
    for i, df in file.groupby('PMID'):
        if len(df['Year'].unique()) == 1:
            for key in year_analysis:
                if df['Year'].unique()[0] >= int(key.split('-')[0]) and df['Year'].unique()[0] < int(key.split('-')[1]):
                    year_analysis[key] +=1
        else:
            print(i, df['Year'].unique())

In [4]:
path = '../trendanalysis_finegrain/'
path_miss = '../trendanalysis_finegrain/missing_data/'
path_fulltext = '../trendanalysis_finegrain/fulltext/'
files = [i for i in os.listdir(path) if 'edited' not in i and '.csv' in i]
analysis_predictions_df = pd.DataFrame()
for file in files:
    analysis_predictions_df = pd.concat([analysis_predictions_df, pd.read_csv(os.path.join(path, file), header=0)])
    analysis_predictions_df = pd.concat([analysis_predictions_df, pd.read_csv(os.path.join(path_miss, file), header=0).drop(['IF'], axis=1)])
    analysis_predictions_df = pd.concat([analysis_predictions_df, pd.read_csv(os.path.join(path_fulltext, file), header=0).drop(['IF'], axis=1)])
    

/home/lanj3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/home/lanj3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':
/home/lanj3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and sil

In [5]:
analysis_predictions_df = analysis_predictions_df[~analysis_predictions_df['CONSORT_item'].isin(['3b', '6b'])]

In [5]:
report_ratio = []
data_df_per_YEAR = analysis_predictions_df[(analysis_predictions_df["Year"] < 2018) & (analysis_predictions_df["Year"] >= 2010)]
for pmid, df_displine_pmid in data_df_per_YEAR.groupby('PMID'):
        report_ratio.append(list(df_displine_pmid.is_reported).count('Y'))
from collections import Counter
Counter(report_ratio)


Counter({5: 11363,
         6: 12591,
         10: 7546,
         8: 10564,
         3: 3841,
         4: 7851,
         7: 12028,
         9: 9030,
         11: 5738,
         0: 635,
         12: 3464,
         13: 1586,
         2: 1693,
         1: 941,
         14: 432,
         15: 61,
         16: 3})

In [6]:
len(report_ratio)

89367

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='scatter_density')
ax.scatter_density(x, y)
ax.set_xlim(-5, 10)
ax.set_ylim(-5, 10)

In [6]:
year_analysis = {"0-1990": [], "1990-1995":[], "1995-2000":[], "2000-2005":[], "2005-2010":[], "2010-2018":[]}

In [16]:
output = []
for year in year_analysis.keys():
    print(year)
    year = year.split("-")
    report_ratio = []
    data_df_per_YEAR = analysis_predictions_df[(analysis_predictions_df["Year"] < int(year[1])) & (analysis_predictions_df["Year"] >= int(year[0]))]
    for pmid, df_displine_pmid in data_df_per_YEAR.groupby('PMID'):
        report_ratio.append(list(df_displine_pmid.is_reported).count('Y')/15)
    mean_item_period = np.mean(report_ratio)
    ratio_list = []
    for i in range(1000):
        ratio_list.append(sum(choices(report_ratio, k=len(report_ratio)))/len(report_ratio))
    cil, ciu=np.quantile(ratio_list, 0.025), np.quantile(ratio_list, 0.975)
    output.append(['-'.join(year), mean_item_period, cil, ciu, mean_item_period - cil, mean_item_period - cil])
#     print(cil, ciu)
#     print(len(report_ratio))
#     print(np.mean(report_ratio)-cil)
#     print('------------------------')

0-1990
1990-1995
1995-2000
2000-2005
2005-2010
2010-2018


In [15]:
pd.DataFrame(output)

,0,1,2,3,4,5,6
0,0-1990,999,0.244308,0.240778,0.247651,0.003530,0.003530
1,1990-1995,999,0.294580,0.290981,0.298179,0.003599,0.003599
2,1995-2000,999,0.360256,0.357320,0.363191,0.002935,0.002935
3,2000-2005,999,0.402244,0.400299,0.404372,0.001945,0.001945
4,2005-2010,999,0.438840,0.437291,0.440437,0.001548,0.001548
5,2010-2018,999,0.474658,0.473423,0.475969,0.001235,0.001235


In [8]:
label_name = ['3a', '4a', '4b', '5', '6a', '7a', '7b', '8a', '8b', '9', '10',
              '11a', '11b', '12a', '12b']


In [26]:
finegrain_item = []

for year in year_analysis.keys():
    print(year)
    year = year.split("-")
    ratio = dict()
    for i in label_name:
        ratio[i] = []
    data_df_per_YEAR = analysis_predictions_df[(analysis_predictions_df["Year"] < int(year[1])) & (analysis_predictions_df["Year"] >= int(year[0]))]
    for item, df_year_item in data_df_per_YEAR.groupby('CONSORT_item'):
        ratio[item].extend(list((df_year_item.is_reported == 'Y').astype(int)))
    for i in ratio:
        mean_item_period = np.mean(ratio[i])
        ratio_list = []
        for j in range(1000):
            ratio_list.append(sum(choices(ratio[i], k=len(ratio[i])))/len(ratio[i]))
        cil, ciu=np.quantile(ratio_list, 0.025), np.quantile(ratio_list, 0.975)
#         error_item_period = mean_item_period - cil
        finegrain_item.append(['-'.join(year), i, mean_item_period, cil, ciu, mean_item_period - cil, ciu - mean_item_period])

0-1990
1990-1995
1995-2000
2000-2005
2005-2010
2010-2018


In [25]:
finegrain_item = []

for year in year_analysis.keys():
    print(year)
    year = year.split("-")
    ratio = dict()
    for i in label_name:
        ratio[i] = []
    data_df_per_YEAR = analysis_predictions_df[(analysis_predictions_df["Year"] < int(year[1])) & (analysis_predictions_df["Year"] >= int(year[0]))]
    for item, df_year_item in data_df_per_YEAR.groupby('CONSORT_item'):
        ratio[item].append(df_year_item[df_year_item.is_reported == 'Y'].shape[0]/df_year_item.shape[0])
    for i in ratio:
        mean_item_period = np.mean(ratio[i])
        ratio_list = []
        for i in range(1000):
            ratio_list.append(sum(choices(ratio[i], k=len(ratio[i])))/len(ratio[i]))
        cil, ciu=np.quantile(ratio_list, 0.025), np.quantile(ratio_list, 0.975))
        error_item_period = mean_item_period - cil
        finegrain_item.append(['-'.join(year), i, mean_item_period, cil, ciu, mean_item_period - cil， mean_item_period - cil])
    year_analysis['-'.join(year)] = ratio

0-1990


KeyError: 0

In [11]:
p_value = []
for item in label_name:
        p_value.append((item, st.ttest_ind(year_analysis['1990-1995'][item], year_analysis['2010-2018'][item])[1]))

In [13]:
pd.DataFrame(p_value)

,0,1
0,3a,1.275947e-07
1,3b,7.440468e-03
2,4a,1.403638e-09
3,4b,1.899291e-10
4,5,4.843604e-06
5,6a,2.465460e-10
6,6b,2.760055e-05
7,7a,1.090652e-09
8,7b,2.954048e-08
9,8a,4.566271e-09


In [24]:
pd.DataFrame(finegrain_item).sort_values(by=[1,0]).to_csv('UTRECHT_analysis_year_finegrain.csv', index=False)

In [28]:
pd.DataFrame(finegrain_item).sort_values(by=[1,0]).to_csv('UTRECHT_analysis_year_finegrain.csv', index=False)